In [ ]:
# 🧪 Step 1: Install dependencies
!pip install -q biopython tqdm

# 🧪 Step 2: Import modules
import os
import pandas as pd
import pickle
from Bio.PDB import PDBParser, Polypeptide
from tqdm import tqdm
from google.colab import drive

# 🧪 Step 3: Mount Google Drive
drive.mount('/content/drive')

# 🧪 Step 4: Set paths
pdb_folder = "/content/drive/MyDrive/RNA-peptide/pdb_files"
input_file = "/content/drive/MyDrive/RNA-peptide/RPI2241.txt"
output_folder = "/content/drive/MyDrive/RNA-peptide/chunks"
os.makedirs(output_folder, exist_ok=True)

# 🧪 Step 5: Load interaction pairs (Tab-delimited: protein_id <tab> rna_id)
with open(input_file, "r") as f:
    lines = f.readlines()

# Strip and split lines
pairs = [line.strip().split('\t') for line in lines if len(line.strip().split('\t')) == 2]
print(f"📊 Total input pairs: {len(pairs)}")

# 🧪 Step 6: Initialize parser
parser = PDBParser(QUIET=True)

# 🧪 Step 7: Process in chunks
chunk_size = 200
for i in range(0, len(pairs), chunk_size):
    chunk_pairs = pairs[i:i+chunk_size]
    results = []
    print(f"\n🔢 Processing chunk {i//chunk_size + 1}/{(len(pairs) + chunk_size - 1)//chunk_size} ({i}–{i+len(chunk_pairs)})")

    for pair in tqdm(chunk_pairs):
        prot_raw, rna_raw = pair
        pdb_id = prot_raw[:4].lower()
        prot_chain = prot_raw[5:]
        rna_chain = rna_raw[5:]
        pdb_file = os.path.join(pdb_folder, f"{pdb_id}.pdb")

        # --- Logging ---
        print(f"\n🔍 {pdb_id.upper()}: Prot[{prot_chain}] RNA[{rna_chain}]")
        if not os.path.exists(pdb_file):
            print(f"❌ Skipping: File {pdb_file} not found.")
            continue

        try:
            structure = parser.get_structure(pdb_id, pdb_file)
            model = structure[0]
            chain_ids = [chain.id for chain in model]
            print(f"✅ Found chains: {chain_ids}")

            from Bio.Data.IUPACData import protein_letters_3to1

            # Extract protein chain
            if prot_chain not in model:
                print(f"❌ Missing protein chain [{prot_chain}]")
                continue
            prot_residues = [res for res in model[prot_chain] if Polypeptide.is_aa(res)]
            three_to_one = protein_letters_3to1.get
            prot_seq = "".join([three_to_one(res.get_resname().capitalize(), 'X') for res in prot_residues])
            print(f"✅ Protein length: {len(prot_seq)}")


            # Extract RNA chain
            if rna_chain not in model:
                print(f"❌ Missing RNA chain [{rna_chain}]")
                continue
            rna_residues = [res for res in model[rna_chain] if not Polypeptide.is_aa(res)]
            rna_seq = "".join([res.resname.strip() for res in rna_residues])
            print(f"✅ RNA length (residues): {len(rna_seq)}")

            # Add to results
            results.append({
                "pdb_id": pdb_id,
                "protein_chain": prot_chain,
                "protein_seq": prot_seq,
                "rna_chain": rna_chain,
                "rna_seq": rna_seq,
            })

        except Exception as e:
            print(f"⚠️ Error parsing {pdb_id}: {e}")
            continue

    # Save chunk
    if results:
        df = pd.DataFrame(results)
        chunk_num = i // chunk_size + 1
        csv_path = os.path.join(output_folder, f"chunk_{chunk_num}.csv")
        pkl_path = os.path.join(output_folder, f"chunk_{chunk_num}.pkl")
        df.to_csv(csv_path, index=False)
        with open(pkl_path, "wb") as f:
            pickle.dump(results, f)
        print(f"✅ Saved {len(results)} pairs to:\n  📄 {csv_path}\n  📦 {pkl_path}")
    else:
        print("⚠️ No valid pairs in this chunk.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📊 Total input pairs: 2242

🔢 Processing chunk 1/12 (0–200)


  0%|          | 0/200 [00:00<?, ?it/s]


🔍 PROT: Prot[in] RNA[]
❌ Skipping: File /content/drive/MyDrive/RNA-peptide/pdb_files/prot.pdb not found.

🔍 1DRZ: Prot[A] RNA[B]
✅ Found chains: ['B', 'A']
✅ Protein length: 95
✅ RNA length (residues): 102

🔍 1VOS: Prot[M] RNA[A]


  2%|▏         | 3/200 [00:00<00:54,  3.58it/s]

✅ Found chains: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'a']
✅ Protein length: 125
✅ RNA length (residues): 1526

🔍 1VOX: Prot[K] RNA[A]


...

✅ Found chains: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'V']
✅ Protein length: 150
✅ RNA length (residues): 1506

🔍 2ZM6: Prot[D] RNA[A]


 98%|█████████▊| 41/42 [00:30<00:00,  1.12it/s]

✅ Found chains: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'V']
✅ Protein length: 208
✅ RNA length (residues): 1506

🔍 2IHX: Prot[A] RNA[B]


100%|██████████| 42/42 [00:31<00:00,  1.33it/s]

✅ Found chains: ['B', 'A']
✅ Protein length: 50
✅ RNA length (residues): 75
✅ Saved 42 pairs to:
  📄 /content/drive/MyDrive/RNA-peptide/chunks/chunk_12.csv
  📦 /content/drive/MyDrive/RNA-peptide/chunks/chunk_12.pkl


In [1]:
from google.colab import drive

# 🧪 Step 1:  Mount Google Drive
drive.mount('/content/drive')
# CSV concatenation
import pandas as pd
import glob

csv_files = sorted(glob.glob("/content/drive/MyDrive/RNA-peptide/chunks/chunk_*.csv"))
df_all = pd.concat([pd.read_csv(f) for f in csv_files])
df_all.to_csv("/content/drive/MyDrive/RNA-peptide/rna_protein_positive_pairs_full.csv", index=False)

# PKL concatenation
import pickle

pkl_files = sorted(glob.glob("/content/drive/MyDrive/RNA-peptide/chunks/chunk_*.pkl"))
all_data = []
for f in pkl_files:
    with open(f, "rb") as infile:
        all_data.extend(pickle.load(infile))
with open("/content/drive/MyDrive/RNA-peptide/rna_protein_positive_pairs_full.pkl", "wb") as outfile:
    pickle.dump(all_data, outfile)


Mounted at /content/drive
